### Data Science Capstone Here we go! 

    Thus begins the start of my final project for the IBM Data Science Professional certificate. I will be providing my code throughout the rest of the notebook for the final project. Enjoy!

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course! Let's do this!!!!! WOO!")

Hello Capstone Project Course! Let's do this!!!!! WOO!


# **Segmenting and Clustering Neighborhoods in Toronto, Canada - Part 1**

   This portion of the assignment requires us to gather, explore, segement and cluster information on neighborhoods in Toronto, Canada. However, unlike in our previous labs we must learn how to extract the necessary information from a source website, in this case wikipedia. All information should be readily available to match the format generated in our lab assignments for New York. This will be broken into three parts.
    

## Loading libraries to extract, explore, segment and cluster information about neighborhoods in Toronto, Canada.

In [207]:
import numpy as np #Library required for vectorized data handling

import pandas as pd #Library required for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #Library required to handle JSON files

from bs4 import BeautifulSoup #utilized for web scraping data

!conda install -c conda-forge geopy=1.49.0 --yes # uncomment this line if you haven't installed the following library for python
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge geocoder --yes
import geocoder

import requests # library to handle requests
import urllib.request

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the following library for python
import folium # map rendering library

print('All libraries are imported!')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - geopy=1.49.0

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


All libraries are imported!


## Obtain and visualize the underlying HTML code within our webpage

In [122]:
#Here we establish the URL that gives us the pertinent information on the Neighborhoods an 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

#parse the HTML data into the BeautifulSoup parse tree format and visualize using prettify
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"Xsl6YApAICkAAD-@iDAAAADA","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":958430791,"wgRevisionId":958430791,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

## Extract the table from our underlying HTML code that contains the data important to us

In [123]:
#Established the specific table we are looking for within the parsed html for our web page
tables = soup.find("table", class_='wikitable sortable')
tables

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<

## Generate lists for each of the columns 

In [102]:
#We define 3 empty list to be established with our 3 columns from our web page
#these columns are Postal Code, Borough and Neighbourhood.

A=[]
B=[]
C=[]

#Creates a loop to find all rows that begin with <tr> and contain at least 3 columns. If both
#stipulations are met we extract the text values within <td> for each element in that row and
#append them to our lists.
for row in tables.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).replace("\n",""))
        B.append(cells[1].find(text=True).replace("\n",""))
        C.append(cells[2].find(text=True).replace("\n",""))

## Create and Clean the dataframe to best suit our needs

In [128]:
#Here we simply define the column headers and apply their respective lists to fill the value below
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

#Finally we exclude any Borough that has a value of 'Not assigned'
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df = df.reset_index(drop=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [130]:
#Returns the number of rows within our dataframe
print("The number of rows in our Toronto, Canada neighbourhoods dataframe is {}. ".format(df.shape[0]))

The number of rows in our Toronto, Canada neighbourhoods dataframe is 103. 


In [238]:
#This establishes our focal point on Toronto for our map
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates for Toronto, Canada are {} latitude and {} longitude".format(latitude, longitude))

The geographical coordinates for Toronto, Canada are 43.6534817 latitude and -79.3839347 longitude


## **Segmenting and Clustering Neighborhoods in Toronto, Canada - Part 2**

   I process the postal code information to provide myself with the latitude and longitude for each postal code in relation to their corresponding Borough and Neighborhood. Ensuring the process is successful by vizualizing with folium.

In [239]:
#Repeat a similar process to extracting our data from wikipedia for utilizing
#geocoder to generate latitude and longitude coordinates from the postal codes.

D=[]
E=[]
for pc, borough, neighborhood in zip(df['Postal Code'], df['Borough'], df['Neighborhood']):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
    lat_lng_coords = g.latlng
    
    lat = lat_lng_coords[0]
    lng = lat_lng_coords[1]
    D.append(lat)
    E.append(lng)

#output the list of corresponding latitude and longitude to new columns in the dataframe.
df['Latitude']=D
df['Longitude']=E
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667481,-79.528953
6,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
7,M3B,North York,Don Mills,43.748900,-79.357220
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


Now that the latitude and longitude data has been obtained, let's visualize this with folium.

In [240]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map indicating the neighborhood
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## **Segmenting and Clustering Neighborhoods in Toronto, Canada - Part 3**

I begin by narrowing our results to just the boroughs that contain Toronto

In [250]:
#Narrow our data to just include Boroughs that contain Toronto
bortor = df[df['Borough'].str.contains("Toronto")]
bortor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
15,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
19,M4E,East Toronto,The Beaches,43.678148,-79.295349


Next, I import the library 'os', as all of my API keys and access tokens are contain within condas environmental variables on my local pc.

In [241]:
import os

CLIENT_ID =  os.environ.get("CLIENT_ID")# your Foursquare ID
CLIENT_SECRET =  os.environ.get("CLIENT_SECRET") # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN")

print('Your credentails:')
print('CLIENT_ID: Are contained within your conda environment variables!' )
print('CLIENT_SECRET: Are contained within your conda environment variables!')

Your credentails:
CLIENT_ID: Are contained within your conda environment variables!
CLIENT_SECRET: Are contained within your conda environment variables!


Let's create a function that obtains the top 100 venues in a radius of 500 meters from our coordinates.

In [253]:
limit=100

def getNearbyVenues(names, latitude, longitude, radius = 500) :
    venues_list = []
    
    for name, lat, long in zip(names, latitude, longitude):
        print(name)
        
        #Generate the API request url
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            ACCESS_TOKEN,
            VERSION,
            lat,
            long,
            radius,
            limit)
        
        #Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevant information on nearby venues
        venues_list.append([(
            name,
            lat,
            long,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)          

Let's now utilize this function to determine the top 100 venues in each of our neighborhoods.

In [254]:
toronto_venues = getNearbyVenues(names=bortor['Neighborhood'],
                                latitude=bortor['Latitude'],
                                longitude=bortor['Longitude']
                                )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [256]:
print(toronto_venues.shape)
toronto_venues.head()

(1736, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


In [313]:
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 246 uniques categories.


## Analyze the Venues in each Neighborhood

In [263]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns = cols)

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bath House,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group rows by neighborhood and take the mean of the frequency of occurrence of each category

In [312]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bath House,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.013158,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.026316,0.000000,0.000000,0.0,0.013158,0.000000,0.000000,0.026316,0.000000,0.00,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.026316,0.00,0.000000,0.00,0.000000,0.026316,0.00,0.000000,0.000000,0.000000,0.02

Here we define a function to sort our venues in descending order.

In [265]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we create a dataframe and display the top 10 venues in each neighborhood.

In [267]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cheese Shop,Bakery,Beer Bar,Clothing Store,Breakfast Spot,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Thrift / Vintage Store,Coffee Shop,Gift Shop,Supermarket,Japanese Restaurant,French Restaurant,North Indian Restaurant,Brewery,Lounge
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Café,Restaurant,Hotel,Thai Restaurant,Steakhouse,Sushi Restaurant,Gym,Salad Place,Seafood Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Coffee Shop,Restaurant,Café,French Restaurant,Bar,Lounge,Italian Restaurant,Speakeasy,Gym / Fitness Center
4,Central Bay Street,Coffee Shop,Sushi Restaurant,Clothing Store,Middle Eastern Restaurant,Restaurant,Café,Bubble Tea Shop,Japanese Restaurant,Tea Room,Italian Restaurant


## Clustering Neighborhoods

We begin by running kmeans to cluster the neighborhood into 8 clusters.

In [302]:
# set number of clusters, increased to 8 to help split up a massive cluster
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = bortor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#
toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged# check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,2,Pub,Athletics & Sports,Café,Bakery,Mexican Restaurant,Food Truck,French Restaurant,Chocolate Shop,Tech Startup,Thai Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,0,Coffee Shop,Sushi Restaurant,Middle Eastern Restaurant,Burger Joint,Yoga Studio,Fried Chicken Joint,Spa,Seafood Restaurant,Burrito Place,Salad Place
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Italian Restaurant,Tea Room,Plaza,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,2,Café,Italian Restaurant,Restaurant,Coffee Shop,Hotel,Breakfast Spot,Gastropub,Cosmetics Shop,Seafood Restaurant,Diner
19,M4E,East Toronto,The Beaches,43.678148,-79.295349,2,Home Service,Trail,Park,Business Service,Rental Car Location,Pub,Coffee Shop,Asian Restaurant,Church,Pizza Place
20,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,2,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cheese Shop,Bakery,Beer Bar,Clothing Store,Breakfast Spot,Pub
24,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653,0,Coffee Shop,Sushi Restaurant,Clothing Store,Middle Eastern Restaurant,Restaurant,Café,Bubble Tea Shop,Japanese Restaurant,Tea Room,Italian Restaurant
25,M6G,Downtown Toronto,Christie,43.668602,-79.420387,3,Grocery Store,Café,Playground,Park,Coffee Shop,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650542,-79.384116,2,Coffee Shop,Café,Salad Place,Gym,Breakfast Spot,American Restaurant,Hotel,Steakhouse,Asian Restaurant,Salon / Barbershop
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.664910,-79.438664,2,Park,Furniture / Home Store,Athletics & Sports,Pool,Café,Bus Line,Smoke Shop,Bank,Bakery,Pharmacy


### Visualization of the clusters

In [303]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the Clusters

Below we can determine the venue categories distinguishing the clusters for our neighborhoods in the Toronto boroughs.

### Cluster 0

In [304]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Middle Eastern Restaurant,Burger Joint,Yoga Studio,Fried Chicken Joint,Spa,Seafood Restaurant,Burrito Place,Salad Place
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Italian Restaurant,Tea Room,Plaza,Ramen Restaurant
24,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Clothing Store,Middle Eastern Restaurant,Restaurant,Café,Bubble Tea Shop,Japanese Restaurant,Tea Room,Italian Restaurant
41,East Toronto,0,Ice Cream Shop,Restaurant,Park,Bus Line,Gym / Fitness Center,Business Service,Grocery Store,Massage Studio,Intersection,Market
75,West Toronto,0,Coffee Shop,Eastern European Restaurant,Café,Gift Shop,Bookstore,Breakfast Spot,Sushi Restaurant,Restaurant,Thai Restaurant,American Restaurant
86,Central Toronto,0,Light Rail Station,Athletics & Sports,Coffee Shop,Health & Beauty Service,Business Service,Skating Rink,Liquor Store,Supermarket,Café,Photography Studio


### Cluster 1

In [305]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,1,Bus Line,Gym / Fitness Center,Lawyer,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


### Cluster 2

In [306]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Pub,Athletics & Sports,Café,Bakery,Mexican Restaurant,Food Truck,French Restaurant,Chocolate Shop,Tech Startup,Thai Restaurant
15,Downtown Toronto,2,Café,Italian Restaurant,Restaurant,Coffee Shop,Hotel,Breakfast Spot,Gastropub,Cosmetics Shop,Seafood Restaurant,Diner
19,East Toronto,2,Home Service,Trail,Park,Business Service,Rental Car Location,Pub,Coffee Shop,Asian Restaurant,Church,Pizza Place
20,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cheese Shop,Bakery,Beer Bar,Clothing Store,Breakfast Spot,Pub
30,Downtown Toronto,2,Coffee Shop,Café,Salad Place,Gym,Breakfast Spot,American Restaurant,Hotel,Steakhouse,Asian Restaurant,Salon / Barbershop
31,West Toronto,2,Park,Furniture / Home Store,Athletics & Sports,Pool,Café,Bus Line,Smoke Shop,Bank,Bakery,Pharmacy
37,West Toronto,2,Bar,Coffee Shop,Men's Store,Boutique,Cocktail Bar,Wine Bar,Vietnamese Restaurant,Asian Restaurant,Restaurant,Bakery
42,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant,Bakery,Steakhouse,Tea Room
43,West Toronto,2,Café,Thrift / Vintage Store,Coffee Shop,Gift Shop,Supermarket,Japanese Restaurant,French Restaurant,North Indian Restaurant,Brewery,Lounge
47,East Toronto,2,Park,Fast Food Restaurant,Pizza Place,Sandwich Place,Gym,Restaurant,Board Shop,Food & Drink Shop,Light Rail Station,Liquor Store


### Cluster 3

In [307]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,3,Grocery Store,Café,Playground,Park,Coffee Shop,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
91,Downtown Toronto,3,Candy Store,Playground,Park,Grocery Store,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service


### Cluster 4

In [308]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Downtown Toronto,4,Harbor / Marina,Theme Park,Farm,Fast Food Restaurant,Park,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service


### Cluster 5

In [309]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,5,Tennis Court,Beer Store,Park,Playground,Gym,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space


### Cluster 6

In [310]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,Central Toronto,6,Spa,Park,Gym Pool,Salon / Barbershop,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm


### Cluster 7

In [311]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,East Toronto,7,Night Market,Baseball Field,Athletics & Sports,Business Service,Government Building,Event Space,Electronics Store,Ethiopian Restaurant,Event Service,Yoga Studio


## Segmeneting and Clustering Neighborhoods in Toronto, Canada - Finished 

This concludes the week 3 assignment of extracting, exploring, segmenting and clustering of Toronto, Canada Neighborhoods provided from information on wikipedia and foursquare.